# Creación del dataset

In [1]:
import pandas as pd

Convertimos el raw_dataset en un pandas dataframe con el formato que nos interesa

In [2]:
data1920 = pd.read_excel('data/raw_dataset.xlsx', sheet_name='Applied Computing', index_col=None, na_values=['No terminado'], usecols = "A:D", skiprows=1)
data1920.dropna(how='all', inplace=True)
data1920.insert(loc=0, column = 'Curso', value = '19/20')
data1920.head()

,Curso,#,Id,Score,Date
0,19/20,838.0,3175.0,8.5,2020-03-30 18:35:15+02
1,19/20,837.0,3209.0,7.5,2020-03-30 17:16:20+02
2,19/20,836.0,3229.0,8.5,2020-03-30 11:23:42+02
3,19/20,835.0,3229.0,NaN,2020-03-30 10:48:41+02
4,19/20,834.0,3229.0,NaN,2020-03-30 10:47:22+02


In [3]:
data1819 = pd.read_excel('data/raw_dataset.xlsx', sheet_name='Applied Computing', index_col=None, na_values=['No terminado'], usecols = "E:H", skiprows=19)
data1819.dropna(how='all', inplace=True)
data1819.insert(loc=0, column = 'Curso', value = '18/19')
data1819.head()

,Curso,#,Id,Score,Date
0,18/19,1175.0,3021.0,9.0,2019-05-21 20:10:52+02
1,18/19,1174.0,2865.0,5.5,2019-05-21 15:35:20+02
2,18/19,1173.0,2851.0,9.0,2019-05-21 15:33:15+02
3,18/19,1172.0,2821.0,6.0,2019-05-21 13:38:42+02
4,18/19,1171.0,2736.0,5.5,2019-05-21 13:37:20+02


In [4]:
data1718 = pd.read_excel('data/raw_dataset.xlsx', sheet_name='Applied Computing', index_col=None, na_values=['No terminado'], usecols = "I:L", skiprows=69)
data1718.dropna(how='all', inplace=True)
data1718.insert(loc=0, column = 'Curso', value = '17/18')
data1718.head()

,Curso,#,Id,Score,Date
0,17/18,1345,2311,8.0,2018-06-01 15:03:04+02
1,17/18,1344,2308,8.0,2018-06-01 15:01:31+02
2,17/18,1343,2276,9.5,2018-06-01 14:51:46+02
3,17/18,1342,2276,8.5,2018-06-01 14:07:59+02
4,17/18,1341,2300,6.0,2018-06-01 14:06:12+02


In [5]:
dataComputing = pd.concat([data1718, data1819, data1920], ignore_index=True)
dataComputing.columns = ['CURSO', 'SEQ', 'ID', 'SCORE', 'DATE']
dataComputing['DATE'] = pd.to_datetime(dataComputing.DATE)
dataComputing.head()

,CURSO,SEQ,ID,SCORE,DATE
0,17/18,1345.0,2311.0,8.0,2018-06-01 15:03:04+02:00
1,17/18,1344.0,2308.0,8.0,2018-06-01 15:01:31+02:00
2,17/18,1343.0,2276.0,9.5,2018-06-01 14:51:46+02:00
3,17/18,1342.0,2276.0,8.5,2018-06-01 14:07:59+02:00
4,17/18,1341.0,2300.0,6.0,2018-06-01 14:06:12+02:00


In [6]:
max_dates = {curso: max(dataComputing[dataComputing.CURSO == curso].DATE) for curso in dataComputing.CURSO.unique()}
max_dates

{'17/18': datetime.datetime(2018, 6, 1, 15, 3, 4, tzinfo=tzoffset(None, 7200)),
 '18/19': datetime.datetime(2019, 5, 21, 20, 10, 52, tzinfo=tzoffset(None, 7200)),
 '19/20': datetime.datetime(2020, 3, 30, 18, 35, 15, tzinfo=tzoffset(None, 7200))}

In [7]:
dataComputing['DUF'] = [(max_dates[curso]-fecha).days for curso, fecha in dataComputing[['CURSO', 'DATE']].values]

In [8]:
dataComputing.head()

,CURSO,SEQ,ID,SCORE,DATE,DUF
0,17/18,1345.0,2311.0,8.0,2018-06-01 15:03:04+02:00,0
1,17/18,1344.0,2308.0,8.0,2018-06-01 15:01:31+02:00,0
2,17/18,1343.0,2276.0,9.5,2018-06-01 14:51:46+02:00,0
3,17/18,1342.0,2276.0,8.5,2018-06-01 14:07:59+02:00,0
4,17/18,1341.0,2300.0,6.0,2018-06-01 14:06:12+02:00,0


# Desarrollo de test para el dataset

In [9]:
[False for curso in dataComputing.CURSO.unique() if curso not in ['17/18', '18/19', '19/20']]

[]

In [10]:
len([curso for curso in dataComputing.CURSO.unique() if curso not in ['17/18', '18/19', '19/20']])


0

In [11]:
len([nota for nota in dataComputing.SCORE.unique() if (nota>10)|(nota<0) ])

0

In [12]:
len([duf for duf in dataComputing.DUF.unique() if (duf<0) ])

0

# Caso 1: agrupación de las notas en las 3 últimas y realizar el clustering

In [13]:
# Eliminamos los "No terminados"
data1 = dataComputing[pd.isnull(dataComputing.SCORE)==False]
# data1.drop(['SEQ'], axis = 1, inplace = True)

In [14]:
data1.head()

,CURSO,SEQ,ID,SCORE,DATE,DUF
0,17/18,1345.0,2311.0,8.0,2018-06-01 15:03:04+02:00,0
1,17/18,1344.0,2308.0,8.0,2018-06-01 15:01:31+02:00,0
2,17/18,1343.0,2276.0,9.5,2018-06-01 14:51:46+02:00,0
3,17/18,1342.0,2276.0,8.5,2018-06-01 14:07:59+02:00,0
4,17/18,1341.0,2300.0,6.0,2018-06-01 14:06:12+02:00,0


In [15]:
data1 = data1.groupby(['ID', 'CURSO'])['SCORE', 'DATE', 'DUF'].apply(lambda x: x.sort_values(['DATE'], ascending=False).head(3))
data1.head(12)

SCORE                       DATE  DUF
ID     CURSO                                           
2213.0 17/18 56     8.0  2018-06-01 11:50:03+02:00    0
             140    6.5  2018-06-01 00:38:06+02:00    0
             188    3.0  2018-05-31 20:31:40+02:00    0
2215.0 17/18 75     8.5  2018-06-01 11:16:11+02:00    0
             135    7.5  2018-06-01 01:42:42+02:00    0
             182   10.0  2018-05-31 20:37:15+02:00    0
2217.0 17/18 464    6.0  2018-05-30 16:08:59+02:00    1
             478    6.0  2018-05-30 13:06:44+02:00    2
             782    7.0  2018-04-27 17:34:46+02:00   34
2218.0 17/18 53     8.5  2018-06-01 11:57:50+02:00    0
             83     7.5  2018-06-01 10:46:05+02:00    0
             149    8.5  2018-06-01 00:08:35+02:00    0

In [16]:
data1.reset_index(drop=False)[['CURSO', 'SCORE', 'DUF']]

,CURSO,SCORE,DUF
0,17/18,8.0,0
1,17/18,6.5,0
2,17/18,3.0,0
3,17/18,8.5,0
4,17/18,7.5,0
...,...,...,...
614,19/20,5.0,8
615,19/20,5.0,10
616,19/20,7.5,3
617,19/20,7.5,4


In [17]:
data1g = data1.reset_index(drop=False)
data1g.head()

,ID,CURSO,level_2,SCORE,DATE,DUF
0,2213.0,17/18,56,8.0,2018-06-01 11:50:03+02:00,0
1,2213.0,17/18,140,6.5,2018-06-01 00:38:06+02:00,0
2,2213.0,17/18,188,3.0,2018-05-31 20:31:40+02:00,0
3,2215.0,17/18,75,8.5,2018-06-01 11:16:11+02:00,0
4,2215.0,17/18,135,7.5,2018-06-01 01:42:42+02:00,0


In [18]:
import plotly.express as px

fig = px.box(data1g, x="CURSO", y="SCORE", color="CURSO")
# fig.write_image("out/caso1/boxplot_score_curso.svg")

fig.show()

In [19]:
!conda install -c plotly plotly-orca